# Instacart Market Basket Analysis

Whether we shop from meticulously planned grocery lists or let whimsy guide our grazing, 
our unique food rituals define who we are. 

Instacart, a grocery ordering and delivery app, aims to make it easy to fill your refrigerator 
and pantry with your personal favorites and staples when you need them. 
After selecting products through the Instacart app, personal shoppers review your order and do the in-store shopping and delivery for you.

We are trying to find answers for multiple questions from the datasets such as 

* Which products will an Instacart consumer purchase again?
* Identifying the duplicate products 
* Identifying all attributes of customer purchases

In [ ]:
# Importing Packages

In [91]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from ast import literal_eval
from sklearn.feature_extraction.text import CountVectorizer

## Data Acquision 

Data used from https://www.kaggle.com/c/instacart-market-basket-analysis/data

In [3]:
aisles = pd.read_csv('Data/aisles.csv')
aisles.head(5)

,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation


In [5]:
departments = pd.read_csv('Data/departments.csv')
departments.head(5)

,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol


In [6]:
orders = pd.read_csv('Data/orders.csv')
orders.head(5)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [7]:
products = pd.read_csv('Data/products.csv')
products.head(5)

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [8]:
order_products =pd.read_csv('Data/order_products__train.csv')
order_products.head(5)

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


In [33]:
orders.head()


,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [94]:
orders['eval_set'].value_counts()

prior    3214874
train     131209
test       75000
Name: eval_set, dtype: int64

In [34]:
orders[orders['order_id']==2539329]

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN


In [36]:
orders['order_id'].nunique(),len(orders)

(3421083, 3421083)

### Joining products and aisles dataset on aisle_id dataset on aisle_id and then with departmensts on department_id

In [12]:
products_asiles = pd.merge(products,aisles,on= 'aisle_id' )

In [19]:
products = pd.merge(products_asiles,departments, on ='department_id')

In [17]:
products

,product_id,product_name,aisle_id,department_id,aisle,department_x,department_y
0,1,Chocolate Sandwich Cookies,61,19,cookies cakes,snacks,snacks
1,78,Nutter Butter Cookie Bites Go-Pak,61,19,cookies cakes,snacks,snacks
2,102,Danish Butter Cookies,61,19,cookies cakes,snacks,snacks
3,172,Gluten Free All Natural Chocolate Chip Cookies,61,19,cookies cakes,snacks,snacks
4,285,Mini Nilla Wafers Munch Pack,61,19,cookies cakes,snacks,snacks
...,...,...,...,...,...,...,...
49683,22827,Organic Black Mission Figs,18,10,bulk dried fruits vegetables,bulk,bulk
49684,28655,Crystallized Ginger Chunks,18,10,bulk dried fruits vegetables,bulk,bulk
49685,30365,Vegetable Chips,18,10,bulk dried fruits vegetables,bulk,bulk
49686,38007,Naturally Sweet Plantain Chips,18,10,bulk dried fruits vegetables,bulk,bulk


### Joining order_products and product table on product_id

In [26]:
orders_products = pd.merge(order_products,products, on ='product_id')
orders_products

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,aisle,department_x,department_y
0,1,49302,1,1,Bulgarian Yogurt,120,16,yogurt,dairy eggs,dairy eggs
1,816049,49302,7,1,Bulgarian Yogurt,120,16,yogurt,dairy eggs,dairy eggs
2,1242203,49302,1,1,Bulgarian Yogurt,120,16,yogurt,dairy eggs,dairy eggs
3,1383349,49302,11,1,Bulgarian Yogurt,120,16,yogurt,dairy eggs,dairy eggs
4,1787378,49302,8,0,Bulgarian Yogurt,120,16,yogurt,dairy eggs,dairy eggs
...,...,...,...,...,...,...,...,...,...,...
1384612,3420011,1528,12,0,Sprinkles Decors,97,13,baking supplies decor,pantry,pantry
1384613,3420084,47935,20,0,Classic Original Lip Balm SPF 12,73,11,facial care,personal care,personal care
1384614,3420084,9491,21,0,Goats Milk & Chai Soap,25,11,soap,personal care,personal care
1384615,3420088,16380,12,0,Stevia Sweetener,97,13,baking supplies decor,pantry,pantry


In [30]:
orders_products['department'] = orders_products['department_x']
orders_products = orders_products.drop(['department_x','department_y'],axis=1)

In [31]:
orders_products

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,aisle,department
0,1,49302,1,1,Bulgarian Yogurt,120,16,yogurt,dairy eggs
1,816049,49302,7,1,Bulgarian Yogurt,120,16,yogurt,dairy eggs
2,1242203,49302,1,1,Bulgarian Yogurt,120,16,yogurt,dairy eggs
3,1383349,49302,11,1,Bulgarian Yogurt,120,16,yogurt,dairy eggs
4,1787378,49302,8,0,Bulgarian Yogurt,120,16,yogurt,dairy eggs
...,...,...,...,...,...,...,...,...,...
1384612,3420011,1528,12,0,Sprinkles Decors,97,13,baking supplies decor,pantry
1384613,3420084,47935,20,0,Classic Original Lip Balm SPF 12,73,11,facial care,personal care
1384614,3420084,9491,21,0,Goats Milk & Chai Soap,25,11,soap,personal care
1384615,3420088,16380,12,0,Stevia Sweetener,97,13,baking supplies decor,pantry


In [37]:
orders.head(5)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [42]:
orders['order_id'].nunique(), len(orders_products), orders_products['order_id'].nunique()

(3421083, 1384617, 131209)

### Creating final merged dataset by joining orders and order_products tables by left joining on order_id

In [80]:
orders_ds = pd.merge(orders,orders_products, on ='order_id', how ='left')
orders_ds

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,aisle,department
0,2539329,1,prior,1,2,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2398795,1,prior,2,3,7,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,473747,1,prior,3,3,12,21.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2254736,1,prior,4,4,7,29.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,431534,1,prior,5,4,15,28.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4674486,272231,206209,train,14,6,14,30.0,6846.0,1.0,1.0,Diet Pepsi Pack,77.0,7.0,soft drinks,beverages
4674487,272231,206209,train,14,6,14,30.0,40603.0,4.0,0.0,Fabric Softener Sheets,75.0,17.0,laundry,household
4674488,272231,206209,train,14,6,14,30.0,37966.0,7.0,0.0,French Baguette Bread,112.0,3.0,bread,bakery
4674489,272231,206209,train,14,6,14,30.0,15655.0,5.0,0.0,Dark Chocolate Mint Snacking Chocolate,45.0,19.0,candy chocolate,snacks


In [81]:
orders_ds['aisle'].unique()

array([nan, 'cream', 'soft drinks', 'milk', 'yogurt', 'popcorn jerky',
       'candy chocolate', 'paper goods', 'nuts seeds dried fruit',
       'cereal', 'packaged cheese', 'fresh fruits', 'frozen meals',
       'lunch meat', 'ice cream ice', 'mint gum', 'soy lactosefree',
       'frozen breads doughs', 'chips pretzels', 'soup broth bouillon',
       'energy granola bars', 'prepared soups salads', 'deodorants',
       'packaged vegetables fruits', 'asian foods', 'fresh vegetables',
       'air fresheners candles', 'honeys syrups nectars', 'coffee',
       'meat counter', 'refrigerated', 'baking ingredients',
       'fresh herbs', 'tortillas flat bread', 'tofu meat alternatives',
       'spices seasonings', 'canned meat seafood', 'breakfast bakery',
       'fresh dips tapenades', 'other creams cheeses', 'bread',
       'water seltzer sparkling water', 'pickled goods olives',
       'frozen appetizers sides', 'buns rolls', 'skin care',
       'oils vinegars', 'frozen produce', 'other', 

In [83]:
orders_ds['aisle'] = orders_ds['aisle'].str.replace(' ','')
orders_ds['aisle-department']= orders_ds['aisle'] +' '+orders_ds['department']

In [86]:
(orders_ds['aisle-department'].unique())

array([nan, 'cream dairy eggs', 'softdrinks beverages', 'milk dairy eggs',
       'yogurt dairy eggs', 'popcornjerky snacks',
       'candychocolate snacks', 'papergoods household',
       'nutsseedsdriedfruit snacks', 'cereal breakfast',
       'packagedcheese dairy eggs', 'freshfruits produce',
       'frozenmeals frozen', 'lunchmeat deli', 'icecreamice frozen',
       'mintgum snacks', 'soylactosefree dairy eggs',
       'frozenbreadsdoughs frozen', 'chipspretzels snacks',
       'soupbrothbouillon canned goods', 'energygranolabars snacks',
       'preparedsoupssalads deli', 'deodorants personal care',
       'packagedvegetablesfruits produce', 'asianfoods international',
       'freshvegetables produce', 'airfreshenerscandles household',
       'honeyssyrupsnectars pantry', 'coffee beverages',
       'meatcounter meat seafood', 'refrigerated beverages',
       'bakingingredients pantry', 'freshherbs produce',
       'tortillasflatbread bakery', 'tofumeatalternatives deli',
       '

## Finding Missins values

In [43]:
orders_ds.isnull().sum()
# eval_set 'prior' and test has NULL values so currently considering train dataset only

order_id                        0
user_id                         0
eval_set                        0
order_number                    0
order_dow                       0
order_hour_of_day               0
days_since_prior_order     206209
product_id                3289874
add_to_cart_order         3289874
reordered                 3289874
product_name              3289874
aisle_id                  3289874
department_id             3289874
aisle                     3289874
department                3289874
dtype: int64

In [49]:
orders_ds.eval_set.value_counts()

prior    3214874
train    1384617
test       75000
Name: eval_set, dtype: int64

In [50]:
orders_ds[orders_ds['eval_set']=='prior']

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,aisle,department
0,2539329,1,prior,1,2,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2398795,1,prior,2,3,7,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,473747,1,prior,3,3,12,21.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2254736,1,prior,4,4,7,29.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,431534,1,prior,5,4,15,28.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4674478,2558525,206209,prior,9,4,15,22.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4674479,2266710,206209,prior,10,5,18,29.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4674480,1854736,206209,prior,11,4,10,30.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4674481,626363,206209,prior,12,1,12,18.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [51]:
orders_ds[orders_ds['eval_set']=='train']

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,aisle,department
10,1187899,1,train,11,4,8,14.0,49235.0,10.0,1.0,Organic Half & Half,53.0,16.0,cream,dairy eggs
11,1187899,1,train,11,4,8,14.0,46149.0,11.0,1.0,Zero Calorie Cola,77.0,7.0,soft drinks,beverages
12,1187899,1,train,11,4,8,14.0,196.0,1.0,1.0,Soda,77.0,7.0,soft drinks,beverages
13,1187899,1,train,11,4,8,14.0,27845.0,9.0,0.0,Organic Whole Milk,84.0,16.0,milk,dairy eggs
14,1187899,1,train,11,4,8,14.0,38928.0,3.0,1.0,0% Greek Strained Yogurt,120.0,16.0,yogurt,dairy eggs
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4674486,272231,206209,train,14,6,14,30.0,6846.0,1.0,1.0,Diet Pepsi Pack,77.0,7.0,soft drinks,beverages
4674487,272231,206209,train,14,6,14,30.0,40603.0,4.0,0.0,Fabric Softener Sheets,75.0,17.0,laundry,household
4674488,272231,206209,train,14,6,14,30.0,37966.0,7.0,0.0,French Baguette Bread,112.0,3.0,bread,bakery
4674489,272231,206209,train,14,6,14,30.0,15655.0,5.0,0.0,Dark Chocolate Mint Snacking Chocolate,45.0,19.0,candy chocolate,snacks


In [69]:

orders_ds['aisle'] = orders_ds['aisle'].str.replace(' ',',')
orders_ds['department'] = orders_ds['department'].str.replace(' ',',')

In [61]:
orders_ds['aisles'].value_counts()

fresh,vegetables              150609
fresh,fruits                  150473
packaged,vegetables,fruits     78493
yogurt                         55240
packaged,cheese                41699
                               ...  
kitchen,supplies                 448
baby,bath,body,care              328
baby,accessories                 306
frozen,juice                     294
beauty                           287
Name: aisles, Length: 134, dtype: int64

In [52]:
orders_ds[orders_ds['eval_set']=='test']

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,aisle,department
78,2774568,3,test,13,5,15,11.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
84,329954,4,test,6,3,12,30.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
101,1528013,6,test,4,3,16,22.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
193,1376945,11,test,8,6,11,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
199,1356845,12,test,6,1,20,30.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4674289,2728930,206202,test,23,2,17,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4674312,350108,206204,test,5,4,14,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4674402,1043943,206206,test,68,0,20,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4674419,2821651,206207,test,17,2,13,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [98]:
orders_ds['user_id'].nunique(),orders_ds['order_id'].nunique()

(206209, 3421083)

In [97]:
len(orders_ds)

4674491

### Final Orders combined dataset 

In [71]:
orders_ds.tail(5)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,aisle,department
4674486,272231,206209,train,14,6,14,30.0,6846.0,1.0,1.0,Diet Pepsi Pack,77.0,7.0,"soft,drinks",beverages
4674487,272231,206209,train,14,6,14,30.0,40603.0,4.0,0.0,Fabric Softener Sheets,75.0,17.0,laundry,household
4674488,272231,206209,train,14,6,14,30.0,37966.0,7.0,0.0,French Baguette Bread,112.0,3.0,bread,bakery
4674489,272231,206209,train,14,6,14,30.0,15655.0,5.0,0.0,Dark Chocolate Mint Snacking Chocolate,45.0,19.0,"candy,chocolate",snacks
4674490,272231,206209,train,14,6,14,30.0,42606.0,6.0,0.0,Phish Food Frozen Yogurt,37.0,1.0,"ice,cream,ice",frozen


## Pivoting the data using product_id as index

In [87]:
orders_ds_train = orders_ds[orders_ds['eval_set']=='train']

In [54]:
len(orders_ds_train)

1384617

In [88]:
# Finding NULLs in the train dataset
orders_ds_train.isnull().sum()

order_id                  0
user_id                   0
eval_set                  0
order_number              0
order_dow                 0
order_hour_of_day         0
days_since_prior_order    0
product_id                0
add_to_cart_order         0
reordered                 0
product_name              0
aisle_id                  0
department_id             0
aisle                     0
department                0
aisle-department          0
dtype: int64

#### Pivoting the data into product_id

In [89]:
products_ds = orders_ds_train.groupby('product_id', as_index=False).agg(set)
products_ds.head(10)

,product_id,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,add_to_cart_order,reordered,product_name,aisle_id,department_id,aisle,department,aisle-department
0,1.0,"{1981184, 2111236, 1708933, 337286, 1639546, 1...","{59648, 1540, 163848, 30345, 143755, 163339, 3...",{train},"{4, 5, 6, 7, 8, 9, 10, 13, 15, 18, 19, 20, 21,...","{0, 1, 2, 3, 4, 5, 6}","{1, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17...","{0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","{1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, ...","{0.0, 1.0}",{Chocolate Sandwich Cookies},{61.0},{19.0},{cookiescakes},{snacks},{cookiescakes snacks}
1,2.0,"{2481009, 1550225, 1455635, 3288051}","{141144, 10252, 27086, 31111}",{train},"{49, 45, 38, 15}","{0, 3, 5}","{21, 11, 13}","{19.0, 11.0, 6.0, 7.0}","{17.0, 26.0, 5.0, 15.0}","{0.0, 1.0}",{All-Seasons Salt},{104.0},{13.0},{spicesseasonings},{pantry},{spicesseasonings pantry}
2,3.0,"{489378, 72425, 2339119, 1176592, 2741526, 218...","{65512, 205789, 1298, 110202, 89179, 83549}",{train},"{4, 6, 9, 12, 16, 22}","{0, 3, 5, 6}","{16, 10, 12, 13}","{7.0, 18.0, 20.0, 24.0, 27.0, 30.0}","{1.0, 13.0, 5.0, 6.0}",{1.0},{Robust Golden Unsweetened Oolong Tea},{94.0},{7.0},{tea},{beverages},{tea beverages}
3,4.0,"{2742787, 459014, 2891043, 918568, 770868, 237...","{85136, 92944, 38808, 150300, 91550, 109342, 1...",{train},"{32, 33, 34, 4, 5, 6, 7, 8, 36, 10, 14, 17, 18}","{0, 1, 2, 3, 4, 5, 6}","{7, 11, 12, 13, 14, 15, 16, 17, 18, 19, 23}","{3.0, 4.0, 5.0, 6.0, 7.0, 11.0, 12.0, 13.0, 14...","{1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, ...","{0.0, 1.0}",{Smart Ones Classic Favorites Mini Rigatoni Wi...,{38.0},{1.0},{frozenmeals},{frozen},{frozenmeals frozen}
4,5.0,{1854678},{160220},{train},{28},{5},{13},{21.0},{5.0},{1.0},{Green Chile Anytime Sauce},{5.0},{13.0},{marinadesmeatpreparation},{pantry},{marinadesmeatpreparation pantry}
5,7.0,{2693316},{5125},{train},{6},{3},{11},{1.0},{11.0},{1.0},{Pure Coconut Water With Orange},{98.0},{7.0},{juicenectars},{beverages},{juicenectars beverages}
6,8.0,"{2584740, 2157381, 3129800, 414824, 2461469, 1...","{13632, 111109, 186311, 106248, 151688, 59246,...",{train},"{4, 5, 6, 8, 9, 15, 16, 18, 21, 86}","{2, 3, 4, 5, 6}","{7, 10, 12, 13, 15, 18, 20}","{2.0, 6.0, 7.0, 9.0, 13.0, 14.0, 16.0, 18.0, 2...","{1.0, 2.0, 5.0, 6.0, 9.0, 11.0, 13.0, 15.0, 18...","{0.0, 1.0}",{Cut Russet Potatoes Steam N' Mash},{116.0},{1.0},{frozenproduce},{frozen},{frozenproduce frozen}
7,9.0,"{1535845, 1225489, 2599832, 324089, 112891}","{92611, 140202, 193710, 28975, 51283}",{train},"{4, 7, 9, 12, 20}","{0, 1, 2, 6}","{10, 12, 16, 17, 19}","{9.0, 13.0, 23.0, 25.0, 30.0}","{16.0, 8.0, 2.0, 1.0}","{0.0, 1.0}",{Light Strawberry Blueberry Yogurt},{120.0},{16.0},{yogurt},{dairy eggs},{yogurt dairy eggs}
8,10.0,"{2718210, 1733634, 631814, 1534989, 977431, 80...","{190978, 71701, 179222, 146967, 114203, 65052,...",{train},"{4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16,...","{0, 1, 2, 3, 4, 5, 6}","{1, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, ...","{0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","{1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, ...","{0.0, 1.0}",{Sparkling Orange Juice & Prickly Pear Beverage},{115.0},{7.0},{waterseltzersparklingwater},{beverages},{waterseltzersparklingwater beverages}
9,11.0,"{2339045, 877269}","{74919, 7903}",{train},"{11, 31}","{4, 5}","{8, 17}","{10.0, 12.0}","{13.0, 14.0}","{0.0, 1.0}",{Peach Mango Juice},{31.0},{7.0},{refrigerated},{beverages},{refrigerated beverages}


In [66]:
products_ds.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39123 entries, 0 to 39122
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   product_id              39123 non-null  float64
 1   order_id                39123 non-null  object 
 2   user_id                 39123 non-null  object 
 3   eval_set                39123 non-null  object 
 4   order_number            39123 non-null  object 
 5   order_dow               39123 non-null  object 
 6   order_hour_of_day       39123 non-null  object 
 7   days_since_prior_order  39123 non-null  object 
 8   add_to_cart_order       39123 non-null  object 
 9   reordered               39123 non-null  object 
 10  product_name            39123 non-null  object 
 11  aisle_id                39123 non-null  object 
 12  department_id           39123 non-null  object 
 13  aisle                   39123 non-null  object 
 14  department              39123 non-null

In [74]:
products_ds['department'].value_counts()

{snacks}             5020
{personal,care}      4314
{pantry}             4107
{beverages}          3526
{frozen}             3414
{dairy,eggs}         3033
{household}          2327
{canned,goods}       1715
{produce}            1533
{dry,goods,pasta}    1526
{bakery}             1313
{deli}               1119
{breakfast}           932
{missing}             905
{international}       883
{babies}              869
{meat,seafood}        777
{pets}                755
{alcohol}             647
{other}               373
{bulk}                 35
Name: department, dtype: int64

In [75]:
products_ds.columns

Index(['product_id', 'order_id', 'user_id', 'eval_set', 'order_number',
       'order_dow', 'order_hour_of_day', 'days_since_prior_order',
       'add_to_cart_order', 'reordered', 'product_name', 'aisle_id',
       'department_id', 'aisle', 'department'],
      dtype='object')

In [90]:
products_ds.head(10)

,product_id,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,add_to_cart_order,reordered,product_name,aisle_id,department_id,aisle,department,aisle-department
0,1.0,"{1981184, 2111236, 1708933, 337286, 1639546, 1...","{59648, 1540, 163848, 30345, 143755, 163339, 3...",{train},"{4, 5, 6, 7, 8, 9, 10, 13, 15, 18, 19, 20, 21,...","{0, 1, 2, 3, 4, 5, 6}","{1, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17...","{0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","{1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, ...","{0.0, 1.0}",{Chocolate Sandwich Cookies},{61.0},{19.0},{cookiescakes},{snacks},{cookiescakes snacks}
1,2.0,"{2481009, 1550225, 1455635, 3288051}","{141144, 10252, 27086, 31111}",{train},"{49, 45, 38, 15}","{0, 3, 5}","{21, 11, 13}","{19.0, 11.0, 6.0, 7.0}","{17.0, 26.0, 5.0, 15.0}","{0.0, 1.0}",{All-Seasons Salt},{104.0},{13.0},{spicesseasonings},{pantry},{spicesseasonings pantry}
2,3.0,"{489378, 72425, 2339119, 1176592, 2741526, 218...","{65512, 205789, 1298, 110202, 89179, 83549}",{train},"{4, 6, 9, 12, 16, 22}","{0, 3, 5, 6}","{16, 10, 12, 13}","{7.0, 18.0, 20.0, 24.0, 27.0, 30.0}","{1.0, 13.0, 5.0, 6.0}",{1.0},{Robust Golden Unsweetened Oolong Tea},{94.0},{7.0},{tea},{beverages},{tea beverages}
3,4.0,"{2742787, 459014, 2891043, 918568, 770868, 237...","{85136, 92944, 38808, 150300, 91550, 109342, 1...",{train},"{32, 33, 34, 4, 5, 6, 7, 8, 36, 10, 14, 17, 18}","{0, 1, 2, 3, 4, 5, 6}","{7, 11, 12, 13, 14, 15, 16, 17, 18, 19, 23}","{3.0, 4.0, 5.0, 6.0, 7.0, 11.0, 12.0, 13.0, 14...","{1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, ...","{0.0, 1.0}",{Smart Ones Classic Favorites Mini Rigatoni Wi...,{38.0},{1.0},{frozenmeals},{frozen},{frozenmeals frozen}
4,5.0,{1854678},{160220},{train},{28},{5},{13},{21.0},{5.0},{1.0},{Green Chile Anytime Sauce},{5.0},{13.0},{marinadesmeatpreparation},{pantry},{marinadesmeatpreparation pantry}
5,7.0,{2693316},{5125},{train},{6},{3},{11},{1.0},{11.0},{1.0},{Pure Coconut Water With Orange},{98.0},{7.0},{juicenectars},{beverages},{juicenectars beverages}
6,8.0,"{2584740, 2157381, 3129800, 414824, 2461469, 1...","{13632, 111109, 186311, 106248, 151688, 59246,...",{train},"{4, 5, 6, 8, 9, 15, 16, 18, 21, 86}","{2, 3, 4, 5, 6}","{7, 10, 12, 13, 15, 18, 20}","{2.0, 6.0, 7.0, 9.0, 13.0, 14.0, 16.0, 18.0, 2...","{1.0, 2.0, 5.0, 6.0, 9.0, 11.0, 13.0, 15.0, 18...","{0.0, 1.0}",{Cut Russet Potatoes Steam N' Mash},{116.0},{1.0},{frozenproduce},{frozen},{frozenproduce frozen}
7,9.0,"{1535845, 1225489, 2599832, 324089, 112891}","{92611, 140202, 193710, 28975, 51283}",{train},"{4, 7, 9, 12, 20}","{0, 1, 2, 6}","{10, 12, 16, 17, 19}","{9.0, 13.0, 23.0, 25.0, 30.0}","{16.0, 8.0, 2.0, 1.0}","{0.0, 1.0}",{Light Strawberry Blueberry Yogurt},{120.0},{16.0},{yogurt},{dairy eggs},{yogurt dairy eggs}
8,10.0,"{2718210, 1733634, 631814, 1534989, 977431, 80...","{190978, 71701, 179222, 146967, 114203, 65052,...",{train},"{4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16,...","{0, 1, 2, 3, 4, 5, 6}","{1, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, ...","{0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","{1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, ...","{0.0, 1.0}",{Sparkling Orange Juice & Prickly Pear Beverage},{115.0},{7.0},{waterseltzersparklingwater},{beverages},{waterseltzersparklingwater beverages}
9,11.0,"{2339045, 877269}","{74919, 7903}",{train},"{11, 31}","{4, 5}","{8, 17}","{10.0, 12.0}","{13.0, 14.0}","{0.0, 1.0}",{Peach Mango Juice},{31.0},{7.0},{refrigerated},{beverages},{refrigerated beverages}


In [99]:
# Loading Orders data into csv
orders_ds.to_csv('orders_data.csv', index=False)

products_ds.to_csv('products_data.csv', index=False)

# Steps Performed

1. Imported Relevant packages
2. Loading data from multiple csv files such as aisles,departments, products, orders, product_orders
3. Merged the data from different dataframes using pd.merge and combine everything into orders_ds dataframe 
4. Data has eval_set(prior,train,test), we are considering only train eval_set for model building
5. Pivoted the orders data based on product_id and merged the other columns for each product_id for further analysis and model building